# Markdown Meeting Notes to Google Docs Converter

This notebook converts markdown-formatted meeting notes into a well-formatted Google Doc.

## Features
- Converts markdown headings to Google Docs heading styles
- Preserves bullet point hierarchy with proper indentation
- Converts markdown checkboxes to actual Google Docs checkboxes
- Highlights @mentions with bold and color formatting
- Handles horizontal rules and footer information

## Step 1: Install Required Libraries

In [ ]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

## Step 2: Import Libraries

In [ ]:
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import re

## Step 3: Authenticate with Google

In [ ]:
# Authenticate and create the Docs API service
auth.authenticate_user()
docs_service = build('docs', 'v1')
print("Authentication successful!")

## Step 4: Define Helper Functions

In [ ]:
def create_google_doc(title, service):
    """
    Create a new Google Doc and return its ID.
    
    Args:
        title (str): Title of the document
        service: Google Docs API service instance
    
    Returns:
        str: Document ID if successful, None otherwise
    """
    try:
        document = service.documents().create(body={'title': title}).execute()
        doc_id = document.get('documentId')
        
        print(f"✓ Created document with ID: {doc_id}")
        print(f"✓ View at: https://docs.google.com/document/d/{doc_id}/edit")
        
        return doc_id
    
    except HttpError as error:
        print(f"✗ An error occurred: {error}")
        return None


def parse_markdown_notes(markdown_text):
    """
    Parse markdown text and generate Google Docs API requests.
    
    Args:
        markdown_text (str): Markdown-formatted text
    
    Returns:
        list: List of Google Docs API requests
    """
    requests = []
    index = 1  # Start after the title
    
    lines = markdown_text.strip().split('\n')
    
    for i, line in enumerate(lines):
        if not line.strip():
            continue
        
        # Main title (# heading)
        if line.startswith('# '):
            text = line[2:].strip()
            requests.append({
                'insertText': {
                    'location': {'index': index},
                    'text': text + '\n'
                }
            })
            requests.append({
                'updateParagraphStyle': {
                    'range': {
                        'startIndex': index,
                        'endIndex': index + len(text) + 1
                    },
                    'paragraphStyle': {
                        'namedStyleType': 'HEADING_1'
                    },
                    'fields': 'namedStyleType'
                }
            })
            index += len(text) + 1
        
        # Section headers (## heading)
        elif line.startswith('## '):
            text = line[3:].strip()
            requests.append({
                'insertText': {
                    'location': {'index': index},
                    'text': text + '\n'
                }
            })
            requests.append({
                'updateParagraphStyle': {
                    'range': {
                        'startIndex': index,
                        'endIndex': index + len(text) + 1
                    },
                    'paragraphStyle': {
                        'namedStyleType': 'HEADING_2'
                    },
                    'fields': 'namedStyleType'
                }
            })
            index += len(text) + 1
        
        # Sub-section headers (### heading)
        elif line.startswith('### '):
            text = line[4:].strip()
            requests.append({
                'insertText': {
                    'location': {'index': index},
                    'text': text + '\n'
                }
            })
            requests.append({
                'updateParagraphStyle': {
                    'range': {
                        'startIndex': index,
                        'endIndex': index + len(text) + 1
                    },
                    'paragraphStyle': {
                        'namedStyleType': 'HEADING_3'
                    },
                    'fields': 'namedStyleType'
                }
            })
            index += len(text) + 1
        
        # Horizontal rule
        elif line.strip() == '---':
            requests.append({
                'insertText': {
                    'location': {'index': index},
                    'text': '─' * 50 + '\n'
                }
            })
            index += 51
        
        # Checkbox items (- [ ] or - [x])
        elif re.match(r'^-\s*\[([ x])\]', line):
            match = re.match(r'^-\s*\[([ x])\]\s*(.+)', line)
            if match:
                text = match.group(2).strip()
                
                # Handle @mentions
                mention_pattern = r'@(\w+)'
                mentions = re.findall(mention_pattern, text)
                
                # Insert the text
                requests.append({
                    'insertText': {
                        'location': {'index': index},
                        'text': text + '\n'
                    }
                })
                
                # Create a checkbox
                requests.append({
                    'createParagraphBullets': {
                        'range': {
                            'startIndex': index,
                            'endIndex': index + len(text) + 1
                        },
                        'bulletPreset': 'BULLET_CHECKBOX'
                    }
                })
                
                # Apply bold and color to @mentions
                if mentions:
                    for mention in mentions:
                        mention_text = f'@{mention}'
                        mention_start = text.find(mention_text)
                        if mention_start != -1:
                            requests.append({
                                'updateTextStyle': {
                                    'range': {
                                        'startIndex': index + mention_start,
                                        'endIndex': index + mention_start + len(mention_text)
                                    },
                                    'textStyle': {
                                        'bold': True,
                                        'foregroundColor': {
                                            'color': {
                                                'rgbColor': {
                                                    'red': 0.2,
                                                    'green': 0.4,
                                                    'blue': 0.8
                                                }
                                            }
                                        }
                                    },
                                    'fields': 'bold,foregroundColor'
                                }
                            })
                
                index += len(text) + 1
        
        # Regular bullet points (* or -)
        elif re.match(r'^[\s]*[\*-]\s+', line):
            # Count indentation level based on leading spaces
            leading_spaces = len(line) - len(line.lstrip())
            indent_level = leading_spaces // 2
            
            
            # Remove leading spaces and bullet marker
            text = re.sub(r'^[\*-]\s+', '', line.lstrip()).strip()
            
            requests.append({
                'insertText': {
                    'location': {'index': index},
                    'text': text + '\n'
                }
            })
            
            # Apply bullet formatting
            requests.append({
                'createParagraphBullets': {
                    'range': {
                        'startIndex': index,
                        'endIndex': index + len(text) + 1
                    },
                    'bulletPreset': 'BULLET_DISC_CIRCLE_SQUARE'
                }
            })
            
            # Apply indentation if needed
            if indent_level > 0:
                requests.append({
                    'updateParagraphStyle': {
                        'range': {
                            'startIndex': index,
                            'endIndex': index + len(text) + 1
                        },
                        'paragraphStyle': {
                            'indentStart': {
                                'magnitude': 36 * indent_level,
                                'unit': 'PT'
                            }
                        },
                        'fields': 'indentStart'
                    }
                })
            
            index += len(text) + 1
        
        # Regular text
        else:
            text = line.strip()
            if text:
                requests.append({
                    'insertText': {
                        'location': {'index': index},
                        'text': text + '\n'
                    }
                })
                index += len(text) + 1
    
    return requests


def convert_markdown_to_google_doc(markdown_text, service, doc_title="Product Team Sync"):
    """
    Main function to convert markdown text to a Google Doc.
    
    Args:
        markdown_text (str): Markdown-formatted text
        service: Google Docs API service instance
        doc_title (str): Title for the document
    
    Returns:
        str: Document ID if successful, None otherwise
    """
    try:
        # Create a new Google Doc
        doc_id = create_google_doc(doc_title, service)
        
        if not doc_id:
            print("✗ Failed to create document")
            return None
        
        # Parse markdown and generate requests
        print("\n✓ Parsing markdown...")
        requests = parse_markdown_notes(markdown_text)
        
        if requests:
            print(f"✓ Generated {len(requests)} formatting requests")
            print("✓ Applying formatting...")
            
            # Batch update the document
            result = service.documents().batchUpdate(
                documentId=doc_id,
                body={'requests': requests}
            ).execute()
            
            print(f"✓ Document updated successfully!")
        
        return doc_id
    
    except HttpError as error:
        print(f"✗ An error occurred: {error}")
        return None

## Step 5: Define Meeting Notes

In [ ]:
# Sample markdown meeting notes
MEETING_NOTES = """# Product Team Sync - May 15, 2023
## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)
## Agenda
### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration
### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues
### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates
## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal
## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders
## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets
---
Meeting recorded by: Sarah Chen
Duration: 45 minutes"""

print("Meeting notes loaded successfully!")
print(f"Total lines: {len(MEETING_NOTES.split(chr(10)))}")

## Step 6: Convert to Google Doc

In [ ]:
print("=" * 60)
print("Starting Markdown to Google Docs Conversion")
print("=" * 60)
print()

doc_id = convert_markdown_to_google_doc(MEETING_NOTES, docs_service)

if doc_id:
    print()
    print("=" * 60)
    print("✓ Conversion completed successfully!")
    print(f"✓ Document URL: https://docs.google.com/document/d/{doc_id}/edit")
    print("=" * 60)
else:
    print()
    print("✗ Conversion failed. Please check the errors above.")

## Optional: Convert Your Own Markdown File

Upload a .md file and convert it to a Google Doc

In [ ]:
# Uncomment and run this cell to upload and convert your own markdown file

# from google.colab import files

# # Upload markdown file
# uploaded = files.upload()

# # Read the uploaded file
# for filename in uploaded.keys():
#     with open(filename, 'r', encoding='utf-8') as f:
#         custom_markdown = f.read()
#     
#     # Extract title from filename (remove .md extension)
#     custom_title = filename.replace('.md', '')
#     
#     # Convert to Google Doc
#     custom_doc_id = convert_markdown_to_google_doc(custom_markdown, docs_service, custom_title)
#     
#     if custom_doc_id:
#         print(f"\n✓ Document created: https://docs.google.com/document/d/{custom_doc_id}/edit")